In [46]:
import matplotlib.pyplot as plt
import seaborn as sns
# Configs
%config InlineBackend.figure_format ='retina'
%matplotlib inline

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn import base, neural_network, neighbors, feature_extraction, datasets, linear_model, metrics, model_selection, preprocessing, svm, ensemble, decomposition, feature_extraction, utils

import keras
import joblib

In [10]:
# Load Bridge helper classes
%run helpers/bridge.ipynb
%run helpers/load-data.ipynb
%run helpers/train-eval-utils.ipynb

In [13]:
# Loads & precomputes all the data, takes about 2.5min
data = load_deals()

In [170]:
# Concatenating features
X = [np.array([h.feature_array() for h in d.hands]).flatten()
     for d in data]

# Winners in a particular strain played from N
y_NT = [d.wnrs[strains.index('NT')]['N'] for d in data]
y_suit = [d.wnrs[strains.index('s')]['N'] for d in data]

In [ ]:
# PREVIOUS PERFORMANCE with old features, smaller dataset

# Features: Globally-defined X
#     Targets: Globally-defined y


#         M: Skl 1layNN #  test MAE: 0.76 - tr MAE: 0.72 - T: 256.4s # 40.2|86.2|97.8
#         M: HistGB     #  test MAE: 0.84 - tr MAE: 0.82 - T:   6.6s # 36.3|83.6|97.0
#         M: LinearRegr #  test MAE: 0.97 - tr MAE: 0.96 - T:   0.7s # 32.3|77.6|94.8


In [ ]:
#   10 iter = 1.5 min
#        M: Skl 1layNN #  test MAE: 0.80 - tr MAE: 0.79 - T: 59.6s # 38.0|84.9|97.5

#    200 iterations = 45 mins
#        M: Skl 1layNN #  test MAE: 0.73 - tr MAE: 0.70 - T: 2753.0s # 41.2|87.9|98.4

# 10 iter = 6 min
#        M: 4 layer 256 #  test MAE: 0.70 - tr MAE: 0.67 - T: 351.3s # 43.0|88.9|98.4
#        M: 7 layer 256 #  test MAE: 0.68 - tr MAE: 0.65 - T: 445.1s # 44.2|89.4|98.5
#           12 layer was same as 7, but worse training time

## results with mask instead
#        M: 7 layer 256 #  test MAE: 0.77 - tr MAE: 0.67 - T: 598.2s # 39.5|85.9|97.6


#    convergence = 44 iterations
#        M: Sklearn-NN #  test MAE: 0.61 - tr MAE: 0.50 - T: 6613.4s # 49.0|91.8|98.8


In [ ]:
# sklearn 2h on NT

# Features: Globally-defined X
#     Targets: Globally-defined y

#         M: Sklearn-NN #  test MAE: 0.61 - tr MAE: 0.50 - T: 6613.4s # 49.0|91.8|98.8
            
            
# sklearn 2h on s
# Features: Globally-defined X
#     Targets: SPADE WINNERS

#         M: Sklearn-NN #  test MAE: 0.45 - tr MAE: 0.37 - T: 5847.6s # 57.4|97.6|99.9



In [ ]:
keras_m = train_eval_dds({'m': seq_model, 'desc': 'MLPNadamTan',
            'b_args': {'activation': 'relu', 'optimizer': keras.optimizers.Nadam(),
                       'layers_no': 6, 'dropout': False},
            'f_args': {'batch_size': 64, 'epochs': 100, 'verbose': 1, 'validation_split': 0.1}}, 
                         targets=[(y_NT, 'NT')], ret_model=True)  # 3min/epoch on CPU

Features: Globally-defined X
    Targets: NT

Train on 303550 samples, validate on 33728 samples
Epoch 1/100
303550/303550 [==============================] - 198s 653us/step - loss: 1.7451 - mean_absolute_error: 1.0073 - val_loss: 1.2809 - val_mean_absolute_error: 0.8853
Epoch 2/100
303550/303550 [==============================] - 116s 381us/step - loss: 1.2299 - mean_absolute_error: 0.8668 - val_loss: 1.1385 - val_mean_absolute_error: 0.8330
Epoch 3/100
303550/303550 [==============================] - 109s 358us/step - loss: 1.1307 - mean_absolute_error: 0.8290 - val_loss: 1.1490 - val_mean_absolute_error: 0.8366
Epoch 4/100
303550/303550 [==============================] - 108s 357us/step - loss: 1.0722 - mean_absolute_error: 0.8059 - val_loss: 1.0520 - val_mean_absolute_error: 0.7938
Epoch 5/100
303550/303550 [==============================] - 108s 357us/step - loss: 1.0331 - mean_absolute_error: 0.7897 - val_loss: 1.0326 - val_mean_absolute_error: 0.7894
Epoch 6/100
303550/303550 [=

In [ ]:
joblib.dump(keras_m, 'models/ftarrays_to_dds_NT_keras.pkl') 

In [ ]:
# 3 epochs

#   nt     M: MLPNadamTan #  test MAE: 0.55 - tr MAE: 0.55 - T: 431.2s # 49.8|95.0|99.8
#    s     M: MLPNadamTan #  test MAE: 0.55 - tr MAE: 0.54 - T: 466.5s # 50.2|95.0|99.7

# 100 epochs
#   nt     M: MLPNadamTan #  test MAE: 0.43 - tr MAE: 0.39 - T: 15855.7s # 59.4|98.1|99.9
#    s     M: MLPNadamTan #  test MAE: 0.44 - tr MAE: 0.40 - T: 13424.1s # 57.8|97.9|99.9



In [161]:
# how to run more epochs
keras_s[0].fit(X, y_suit, validation_split=0.1, epochs=1)

Train on 379438 samples, validate on 42160 samples
Epoch 1/1
379438/379438 [==============================] - 320s 844us/step - loss: 0.3623 - mean_absolute_error: 0.4748 - val_loss: 0.3662 - val_mean_absolute_error: 0.4780


In [ ]:

import matplotlib.pyplot as plt

history = keras_m[0].history

# Plot training & validation accuracy values
plt.plot(history.history['val_mean_absolute_error'])
plt.plot(history.history['mean_absolute_error'])
plt.title('Model accuracy')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.legend(['validation', 'train'], loc='upper left')
plt.show()
